In [116]:
import pandas as pd
import numpy as np
import csv
import re
import sys

In [117]:
names = ['Text']
df = pd.read_table('samp.en', header=None, delimiter=None, names=names)

/home/flame/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [118]:
df.head()

,Text
0,Given that there is now widespread agreement o...
1,"DM Digital Television, a TV company in my regi..."
2,Biol.
3,"As a rule, privatization agreements contain no..."
4,"Leo , this is Cookie Monster ."


In [119]:
df['lang']=0

In [120]:
print(len(df))
df.head()

18375


,Text,lang
0,Given that there is now widespread agreement o...,0
1,"DM Digital Television, a TV company in my regi...",0
2,Biol.,0
3,"As a rule, privatization agreements contain no...",0
4,"Leo , this is Cookie Monster .",0


In [121]:
names = ['Text']
df1 = pd.read_table('samp.es', header=None, delimiter=None, names=names)
df1['lang']=1

names = ['Text']
df2 = pd.read_table('samp.pt', header=None, delimiter=None, names=names)
df2['lang']=2

/home/flame/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/home/flame/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [122]:
print(len(df1))
df1.head()

18782


,Text,lang
0,"Aminta, un humilde pastor, tropieza con ellas,...",1
1,La primera conferencia de las Partes encargada...,1
2,La posición de Israel tiene consecuencias que ...,1
3,Los medicamentos no se deben tirar por los des...,1
4,- el apartado 3 del artículo 3 bis y el artícu...,1


In [123]:
print(len(df2))
df2.head()

19126


,Text,lang
0,A realização extensiva de controlos parece ser...,2
1,"Colbie logo conheceu o produtor Mikal Blue, qu...",2
2,Parte II,2
3,Este relatório deve ser considerado à luz dess...,2
4,Já tive ocasião de dizer que o presente debate...,2


In [124]:
dt = [df, df1, df2]
df = pd.concat(dt)

In [125]:
print(len(df))
df.head()

56283


,Text,lang
0,Given that there is now widespread agreement o...,0
1,"DM Digital Television, a TV company in my regi...",0
2,Biol.,0
3,"As a rule, privatization agreements contain no...",0
4,"Leo , this is Cookie Monster .",0


In [126]:
df = df.sample(frac=1).reset_index(drop=True)

In [127]:
df.head()

,Text,lang
0,Condena a los abusos sexuales durante conflict...,1
1,Impor támos praticamente um milhão de bezerros...,2
2,Final consumption expenditure of resident and ...,0
3,"(14) OJ L 145, 19.6.1996, p.",0
4,The optical classification scheme for seawater...,0


In [128]:
len(df)

56283

In [129]:
df['lang'].value_counts()

2    19126
1    18782
0    18375
Name: lang, dtype: int64

In [130]:
# Preprocessing of the text, took care of basic pre-processing task only.

def processSent(sent):
    # To lowercase
    sent = sent.lower()
    #remove @username
    sent = re.sub(r'@(\w+)','',sent)
    
    # Remove hashtags
    sent = re.sub(r'#(\w+)', '', sent)
    
    # Remove Punctuation and split 's, 't, 've with a space for filter    
    sent = re.sub(r"[-()\"«/;:<>{}`+=~|.!&?',]", "", sent)
    
    # remove the numerical values
    sent = re.sub(r"(\s\d+)","",sent) 
    
    # Remove HTML special entities (e.g. &amp;)
    sent = re.sub(r'\&\w*;', '', sent)
    
    # Remove tickers
    sent = re.sub(r'\$\w*', '', sent)
    
    # Remove hyperlinks
    sent = re.sub(r'https?:\/\/.*\/\w*', '', sent)
   
    # Remove whitespace (including new line characters)
    sent = re.sub(r'\s\s+', ' ', sent)
   
    # Remove single space remaining at the front of the sent.
    sent = sent.lstrip(' ') 
    
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    sent = ''.join(c for c in sent if c <= '\uFFFF') 
    
    return sent

In [131]:
data = df.copy()
data['Text'] = data['Text'].apply(processSent)
print(data.head())

                                                Text  lang
0  condena a los abusos sexuales durante conflict...     1
1  impor támos praticamente um milhão de bezerros...     2
2  final consumption expenditure of resident and ...     0
3                                          14 oj l p     0
4  the optical classification scheme for seawater...     0


In [132]:
names = ['Text']
dft = pd.read_table('langid_langid.test', header=None, delimiter=None, names=names)
print(len(dft))
dft.head()

/home/flame/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


2433


,Text
0,"Com o regresso de Medane e Menad, o treinador ..."
1,"Sin embargo, debe tenerse en cuenta cuando se ..."
2,The updated version of the State Aid Guideline...
3,"For that purpose, EFTA States may notify rescu..."
4,"Batatas (boniatos, camotes), excepto las fresc..."


In [133]:
dft['Text'] = dft['Text'].apply(processSent)

In [134]:
dft.head()

,Text
0,com o regresso de medane e menad o treinador j...
1,sin embargo debe tenerse en cuenta cuando se c...
2,the updated version of the state aid guideline...
3,for that purpose efta states may notify rescue...
4,batatas boniatos camotes excepto las frescas e...


In [135]:


from sklearn.feature_extraction.text import TfidfVectorizer
train = list(data['Text'])
test = list(dft['Text'])
corpus = train+test
tfidf = TfidfVectorizer(max_features = 6000) 
tfidf.fit(corpus)
tfidf_features_train = tfidf.transform(train)
tfidf_features_test = tfidf.transform(test)

print(tfidf_features.shape)

(56283, 6000)


In [136]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support


# Converting the labels from strings to binary (if we have string 'lang' labels)
label = data['lang']
le = LabelEncoder()
le.fit(label)
label = le.transform(label)


import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

# # Taking 80/20 train test split
# train_percent = 0.8
# train_cutoff = int(np.floor(train_percent*len(data['Text']) ) )
# X_train,X_test, Y_train, Y_test = train_test_split(tfidf_features, label, test_size=0.2, random_state=42)
X_train = tfidf_features_train
Y_train = label

X_test = tfidf_features_test

# X_train,X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print("Train: ", (X_train.shape), (Y_train.shape))
print("Test: ", (X_test.shape), (Y_test.shape))
# print("Val: ", (X_val.shape))

Train:  (56283, 6000) (56283,)
Test:  (2433, 6000) (11257,)


In [137]:


tfidf_model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# tfidf_model = MultinomialNB()  
tfidf_model.fit(X_train, Y_train)
tfidf_prediction = tfidf_model.predict(X_test)


# results = pd.DataFrame(index = ['Word Embedding', 'TF-IDF'], 
#           columns = ['Precision', 'Recall', 'F1 score', 'support']
#           )
# results.loc['TF-IDF'] = precision_recall_fscore_support(
#           Y_test, 
#           tfidf_prediction, 
#           average = None
#           )

/home/flame/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


ValueError: Found input variables with inconsistent numbers of samples: [11257, 2433]

In [138]:
# Predicted result
tfidf_prediction

array([2, 1, 0, ..., 1, 2, 1])

In [139]:
np.savetxt('test.txt', tfidf_prediction)

### This all below code was when we trained and tested on our own data ###

In [ ]:
print(results.loc['TF-IDF'])
Y_pred = tfidf_prediction

# k = 11257
k = 2433

P1 = np.zeros((k,3), dtype=np.float32)
P2 = np.zeros((k,3), dtype=np.float32)

for i in range(k):
    P1[i][int(Y_test[i])]=1.0
    P2[i][int(Y_pred[i])]=1.0

print(P1, P2)

In [ ]:
P1 = np.argmax(P1, axis=1)
for ix in range(3):
    print(ix, confusion_matrix(np.argmax(P2, axis=1), P1)[ix].sum())
cm = confusion_matrix(np.argmax(P2, axis=1), P1)
print(cm)

import seaborn as sn
import matplotlib.pyplot as plt
 
df_cm = pd.DataFrame(cm, range(3), range(3))
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True)
sn.set_context("poster")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
# plt.savefig('gdrive/My Drive/PolEval/confusionMatrix2.png')
plt.show()

In [ ]:
target_names = ['En', 'Es', 'Pt']
print(classification_report(np.argmax(P2, axis=1), P1, target_names=target_names))
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(P2, axis=1), P1, normalize=True))